In [1]:
# Import python packages
import os
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [2]:
# Can uncomment to install R packages as needed
#utils = importr('utils')
#utils.install_packages('nflreadr')
#utils.install_packages('ffscrapr')

In [3]:
# Import R packages
ffscrapr = importr('ffscrapr')

In [4]:
# Config for the calculation
mfl_id = 60206  #MFL id for league
seasons = [2023,2022,2021,2020]  #Seasons to collect data for
max_week = 17  #Maximum week number to include in calculation
min_games = 8
save_label = 'Analytics_Dynasty_League_2024'  #Where to save results

load_data = True

In [36]:
# Map from teams to conferences
conf_dic = {
    'Arizona Cardinals':'NFC',
    'Atlanta Falcons':'NFC',
    'Baltimore Ravens':'AFC',
    'Buffalo Bills':'AFC',
    'Carolina Panthers':'NFC',
    'Chicago Bears':'NFC',
    'Cincinnati Bengals':'AFC',
    'Cleveland Browns':'AFC',
    'Dallas Cowboys':'NFC',
    'Denver Broncos':'AFC',
    'Detroit Lions':'NFC',
    'Green Bay Packers':'NFC',
    'Houston Texans':'AFC',
    'Indianapolis Colts':'AFC',
    'Jacksonville Jaguars':'AFC',
    'Kansas City Chiefs':'AFC',
    'Las Vegas Raiders':'AFC',
    'Los Angeles Chargers':'AFC',
    'Los Angeles Rams':'NFC',
    'Miami Dolphins':'AFC',
    'Minnesota Vikings':'NFC',
    'New England Patriots':'AFC',
    'New Orleans Saints':'NFC',
    'New York Giants':'NFC',
    'New York Jets':'AFC',
    'Philadelphia Eagles':'NFC',
    'Pittsburgh Steelers':'AFC',
    'San Francisco 49ers':'NFC',
    'Seattle Seahawks':'NFC',
    'Tampa Bay Buccaneers':'NFC',
    'Tennessee Titans':'AFC',
    'Washington Commanders':'NFC',
    'Washington Football Team':'NFC',
}

In [37]:
# Map from positions to rank floors
rank_floor_dic = {
    'CB':36, 
    'DE':40, 
    'DT':36, 
    'LB':40, 
    'PK':16, 
    'PN':16, 
    'QB':16, 
    'RB':28, 
    'S':40, 
    'TE':16, 
    'WR':52,
}

In [9]:
# Check whether or not to load roster and playerscore data from disk
if load_data and os.path.isfile(f'{save_label}_Rosters.csv') and os.path.isfile(f'{save_label}_PlayerScores.csv'):
    # Will load roster and playerscore data from disk
    print('Loading roster and playerscore data from disk')
    
    rosters_df = pd.read_csv(f'{save_label}_Rosters.csv')
    playerscores_df = pd.read_csv(f'{save_label}_PlayerScores.csv')

else:
    # Will scrape roster  and playerscore data from MFL
    print('Scraping roster and playerscore data from MFL')
    
    # Dict to store data for each season
    rosters_df = {}
    playerscores_df = {}
    
    for s in seasons:
        # Scrape player positions from MFL and convert to pandas df
        mfl = ffscrapr.mfl_connect(season=s, league_id=mfl_id, rate_limit_number=1, rate_limit_seconds=6)
        
        rosters_df_r = ffscrapr.ff_rosters(mfl)
        playerscores_df_r = ffscrapr.ff_playerscores(mfl, season=s, week=[i+1 for i in range(max_week)])
        
        with (ro.default_converter + pandas2ri.converter).context():
            rosters_df[s] = ro.conversion.get_conversion().rpy2py(rosters_df_r)
            playerscores_df[s] = ro.conversion.get_conversion().rpy2py(playerscores_df_r)
    
        # Cleanup some of the data
        rosters_df[s]['season'] = s
        rosters_df[s]['player_id'] = rosters_df[s]['player_id'].astype(int)
    
        playerscores_df[s]['season'] = playerscores_df[s]['season'].astype(int)
        playerscores_df[s]['week'] = playerscores_df[s]['week'].astype(int)
        playerscores_df[s]['player_id'] = playerscores_df[s]['player_id'].astype(int)
        playerscores_df[s]['points'] = playerscores_df[s]['points'].astype(float)
        playerscores_df[s] = playerscores_df[s].drop('is_available', axis=1)
    
    # Merge seasons to single df
    rosters_df = pd.concat(rosters_df, ignore_index=True)
    playerscores_df = pd.concat(playerscores_df, ignore_index=True)
    
    # Sort by for cleaner display
    rosters_df = rosters_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).reset_index()
    playerscores_df = playerscores_df.set_index(['player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True]).reset_index()
    
    # Save a copy of the roster and playerscore dfs
    rosters_df.to_csv(f'{save_label}_Rosters.csv', index=False)
    playerscores_df.to_csv(f'{save_label}_PlayerScores.csv', index=False)

Loading roster and playerscore data from disk


In [10]:
rosters_df

,player_id,season,franchise_id,franchise_name,player_name,pos,team,age,salary,contract_years,contractInfo,roster_status,draft_year,draft_round
0,4925,2020,3,Philadelphia Eagles,"Brees, Drew",QB,NOS,45.2,22.36,1.0,2020 NEFT,ROSTER,2001,2
1,4925,2020,24,Pittsburgh Steelers,"Brees, Drew",QB,NOS,45.2,15.50,2.0,2020 UFA,ROSTER,2001,2
2,5848,2020,14,Los Angeles Rams,"Brady, Tom",QB,TBB,46.6,22.36,1.0,2020 NEFT,ROSTER,2000,6
3,5848,2020,32,Los Angeles Chargers,"Brady, Tom",QB,TBB,46.6,22.70,1.0,2020 UFA,ROSTER,2000,6
4,5848,2021,4,Washington Football Team,"Brady, Tom",QB,TBB,46.6,15.00,2.0,2021 UFA,ROSTER,2000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,16441,2023,17,Buffalo Bills,"DeVito, Tommy",QB,NYG,25.6,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_
5979,16444,2023,17,Buffalo Bills,"Grupe, Blake",PK,NOS,25.3,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_
5980,16457,2023,2,New York Giants,"Izien, Christian",CB,TBB,23.8,1.50,1.0,2023 UFA,ROSTER,2023,NA_character_
5981,16458,2023,17,Buffalo Bills,"Hedley, Lou",PN,NOS,30.7,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_


In [11]:
playerscores_df

,player_id,season,week,player_name,pos,team,points
0,4925,2020,1,"Brees, Drew",QB,NOS,19.7
1,4925,2020,2,"Brees, Drew",QB,NOS,39.9
2,4925,2020,3,"Brees, Drew",QB,NOS,44.5
3,4925,2020,4,"Brees, Drew",QB,NOS,31.5
4,4925,2020,5,"Brees, Drew",QB,NOS,36.8
...,...,...,...,...,...,...,...
77650,16560,2023,16,"Maddox-Williams, Tyreek",LB,ARI,0.0
77651,16560,2023,17,"Maddox-Williams, Tyreek",LB,ARI,0.0
77652,16562,2023,16,"Williams, Jaylin",CB,MIN,-1.0
77653,16562,2023,17,"Williams, Jaylin",CB,MIN,0.0


In [47]:
# Check whether or not to load contract data from disk
if False and load_data and os.path.isfile(f'{save_label}_Contracts.csv'):
    # Will load contract data from disk
    print('Loading contract data from disk')
    
    contracts_df = pd.read_csv(f'{save_label}_Contracts.csv')

else:
    # Will calculate contract data from roster and playerscore data
    print('Calculating contract data from roster and playerscore data')

    # Downselect players to those having enough qualifying games
    playerscores_df = playerscores_df.set_index(['player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])
    playerscores_df['num_games'] = playerscores_df['points'].groupby(['player_id','season']).count()
    
    tmp_df = playerscores_df[ playerscores_df['num_games'] >= min_games ]
    
    playerscores_df = playerscores_df.reset_index()
    
    # Aggregate and rank qualifying player scores by position
    contracts_df = tmp_df.groupby(['player_id','season']).head(1).droplevel(2).drop('points', axis=1)
    contracts_df['tot_pts'] = tmp_df['points'].groupby(['player_id','season']).sum()
    contracts_df['avg_pts'] = tmp_df['points'].groupby(['player_id','season']).mean()
    
    contracts_df = contracts_df.reset_index().set_index(['pos','season']).sort_index(level=[0,1],ascending=[True,True])
    
    contracts_df['tot_pts_rank'] = contracts_df['tot_pts'].groupby(['pos','season']).rank(method='average',ascending=False).values
    contracts_df['avg_pts_rank'] = contracts_df['avg_pts'].groupby(['pos','season']).rank(method='average',ascending=False).values
    contracts_df['floor_pts_rank'] = contracts_df.index.get_level_values(0).map(lambda x: rank_floor_dic[x])
    
    contracts_df = contracts_df.reset_index().set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True])
    
    # Merge in aggregated playerscores to roster_df
    contracts_df = rosters_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).merge(contracts_df[['tot_pts','tot_pts_rank','avg_pts','avg_pts_rank','floor_pts_rank']], how='left', left_index=True, right_index=True)
    contracts_df['conf'] = contracts_df['franchise_name'].map(lambda x: conf_dic[x])
    
    # Rank the salaries by position
    contracts_df = contracts_df.reset_index().set_index(['pos','season']).sort_index(level=[0,1],ascending=[True,True])
    contracts_df['salary_rank'] = contracts_df['salary'].groupby(['pos','season']).rank(method='first',ascending=False).values
    contracts_df = contracts_df.reset_index()
    
    # Sort by for cleaner display
    contracts_df = contracts_df.set_index(['pos','season','salary_rank','conf']).sort_index(level=[0,1,2,3],ascending=[True,True,True,True]).reset_index()
    
    # Save a copy of the contract df
    contracts_df.to_csv(f'{save_label}_Contracts.csv', index=False)

Calculating contract data from roster and playerscore data


/home/ryanl/.local/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [48]:
contracts_df

,pos,season,salary_rank,conf,player_id,franchise_id,franchise_name,player_name,team,age,...,contract_years,contractInfo,roster_status,draft_year,draft_round,tot_pts,tot_pts_rank,avg_pts,avg_pts_rank,floor_pts_rank
0,CB,2020,1.0,AFC,12245,24,Pittsburgh Steelers,"Peters, Marcus",BAL,31.2,...,1.0,Inaugural,ROSTER,2015,1,120.9,20.0,9.300000,15.0,36.0
1,CB,2020,2.0,NFC,12245,3,Philadelphia Eagles,"Peters, Marcus",BAL,31.2,...,1.0,2020 UFA,ROSTER,2015,1,120.9,20.0,9.300000,15.0,36.0
2,CB,2020,3.0,NFC,13249,9,Atlanta Falcons,"White, Tre'Davious",BUF,29.1,...,1.0,2020 NEFT,ROSTER,2017,1,130.5,13.0,8.700000,21.0,36.0
3,CB,2020,4.0,AFC,10789,22,Cincinnati Bengals,"Gilmore, Stephon",NEP,33.5,...,1.0,2020 TT,ROSTER,2012,1,54.7,100.0,4.972727,92.0,36.0
4,CB,2020,5.0,AFC,11764,29,Denver Broncos,"Fuller, Kyle",CHI,32.1,...,1.0,2020 UFA,ROSTER,2014,1,91.4,43.0,5.712500,73.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,WR,2023,237.0,AFC,16199,27,Jacksonville Jaguars,"Palmer, Trey",TBB,22.9,...,3.0,2023 UDFA,ROSTER,2023,6,37.4,100.0,2.337500,112.0,52.0
5979,WR,2023,238.0,AFC,16201,20,New York Jets,"Hutchinson, Xavier",HOU,23.8,...,3.0,2023 UDFA,ROSTER,2023,6,14.6,142.0,0.973333,146.0,52.0
5980,WR,2023,239.0,NFC,16207,6,Detroit Lions,"Iosivas, Andrei",CIN,24.4,...,3.0,2023 UDFA,ROSTER,2023,6,18.0,132.0,1.200000,139.0,52.0
5981,WR,2023,240.0,AFC,16299,29,Denver Broncos,"Davis, Derius",LAC,23.5,...,3.0,2023 UDFA,ROSTER,2023,4,24.4,121.0,1.525000,129.0,52.0


In [85]:
###
def CalculateSalary(df, player_name, player_position, player_conference, contract_season):
    # Function that returns extension salary for specified player
    player_df = df[ (df['player_name'] == player_name) & (df['pos'] == player_position) & (df['conf'] == player_conference) ].tail(2)
    position_df = df[ (df['pos'] == player_position) & (df['season'] == contract_season) ].sort_values('salary_rank')
    salaries = position_df['salary'].values
    
    min_tot_pts_rank = player_df['tot_pts_rank'].min()
    min_avg_pts_rank = player_df['avg_pts_rank'].min()
    min_floor_pts_rank = player_df['floor_pts_rank'].min()

    min_pts_rank = min(min_tot_pts_rank,min_avg_pts_rank,min_floor_pts_rank)
    
    sal_eval_rank_high = int(np.round(2*min_pts_rank - 3))
    sal_eval_rank_low = int(np.round(2*min_pts_rank - 2))

    sal_eval_high = salaries[sal_eval_rank_high - 1]
    sal_eval_low = salaries[sal_eval_rank_low - 1]
    
    return np.round(1.1*(0.5*sal_eval_high + 0.5*sal_eval_low), 2)

In [86]:
CalculateSalary(contracts_df, 'Goff, Jared', 'QB', 'NFC', 2023)

23.47